# 10

In [ ]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        m, n = len(s), len(p)
        dp = [[False] * (n + 1) for _ in range(m + 1)]
        dp[0][0] = True  # Empty pattern matches empty string
        
        # Fill first row (case when s is empty)
        for j in range(1, n + 1):
            if p[j-1] == '*':
                dp[0][j] = dp[0][j-2]
        
        # Fill DP table
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if p[j-1] == '.' or p[j-1] == s[i-1]:
                    dp[i][j] = dp[i-1][j-1]
                elif p[j-1] == '*':
                    # Check zero occurrence (ignore the previous character and *)
                    dp[i][j] = dp[i][j-2]
                    # Check if the previous character matches
                    if j >= 2 and (p[j-2] == s[i-1] or p[j-2] == '.'):
                        dp[i][j] |= dp[i-1][j]
        return dp[m][n]
    
s = "aaa"
p = "ab*a"
Solution().isMatch(s=s,p=p)

False

# 3343

In [ ]:
class Solution(object):
    def countBalancedPermutations(self, num):
        modulo = 10**9+7
        len_s = len(num)
        
        total = sum(int(c) for c in num)
        if total % 2:
            return 0
        
        fact = [1]*(len_s+1)
        inv = [1]*(len_s+1)
        invFact = [1]*(len_s+1)
        
        for i in range(1,len_s+1):
            fact[i] = fact[i-1]*i % modulo
            
        for i in range(2,len_s+1):
            inv[i] = modulo - (modulo//i)*inv[modulo%i] % modulo
            
        for i in range(1,len_s+1):
            invFact[i] = invFact[i-1]*inv[i] % modulo
        
        halfSum = total//2
        halfLen = len_s//2
        dp = [[0]*(halfLen+1) for _ in range(halfSum+1)]
        dp[0][0] = 1
        digits = [0]*10
        
        for c in num:
            d = int(c)
            digits[d] += 1
            for i in range(halfSum, d-1, -1):
                for j in range(halfLen, 0, -1):
                    dp[i][j] = (dp[i][j] + dp[i-d][j-1]) % modulo
                    
        res = dp[halfSum][halfLen]
        res = res * fact[halfLen] % modulo * fact[len_s-halfLen] % modulo
        for cnt in digits: res = res * invFact[cnt] % modulo
        
        return res
        

num = "2073365529"
Solution().countBalancedPermutations(num)

39600

# 3335

In [ ]:
class Solution:
    def lengthAfterTransformations(self, s: str, t: int) -> int:
        MOD = 10**9 + 7
        counts = [0] * 26
        
        for c in s:
            counts[ord(c) - ord('a')] += 1
        
        for _ in range(t):
            new_counts = [0] * 26
            z_count = counts[25]  # Count of 'z'
            
            new_counts[0] = z_count  # 'z' becomes 'a'
            new_counts[1] = (counts[0] + z_count) % MOD  # 'a' becomes 'b' and 'z' becomes 'b'
            
            for i in range(2, 26):
                new_counts[i] = counts[i - 1]  # Shift other characters
            
            for i in range(26):
                new_counts[i] %= MOD
            
            counts = new_counts
        
        return sum(counts) % MOD

# 3337

работает. Но можно лучше

In [1]:
from typing import List
class Solution:
    def lengthAfterTransformations(self, s: str, t: int, nums: List[int]) -> int:
        MOD = 10**9 + 7
        counts = [0] * 26
        
        for c in s:
            counts[ord(c) - ord('a')] += 1
        
        for _ in range(t):
            new_counts = [0] * 26
            
            

            
            for i in range(26):
                    for j in range(nums[i - 1]):
                        new_counts[(i+j) % 26] += counts[i - 1]          
            
            
            for i in range(26):
                new_counts[i] %= MOD
            
            counts = new_counts
        
        return sum(counts) % MOD
assert Solution().lengthAfterTransformations("abcyy",2,[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2]) == 7
assert Solution().lengthAfterTransformations("azbk",1,[2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]) == 8

Должно быть лучше. Расписал [здесь](https://leetcode.com/problems/total-characters-in-string-after-transformations-ii/solutions/6743005/use-matrices-i-beg-you)

In [ ]:
from typing import List
class Solution:
    def lengthAfterTransformations(self, s: str, t: int, nums: List[int]) -> int:
        MOD = 10**9 + 7
        counts = [0] * 26
        for c in s:
            counts[ord(c) - ord('a')] += 1
        
        # Build transition matrix T
        T = [[0] * 26 for _ in range(26)]
        
        for k in range(26):
            L = nums[k]
            for x in range(26):
                d = (x - k) % 26
                if d == 0:
                    d = 26
                if d > L:
                    cnt = 0
                else:
                    cnt = (L - d) // 26 + 1
                T[x][k] = cnt % MOD
        
        # Matrix exponentiation helper functions
        def multiply(A, B):
            res = [[0] * 26 for _ in range(26)]
            for i in range(26):
                for k in range(26):
                    if not A[i][k]:
                        continue
                    for j in range(26):
                        res[i][j] = (res[i][j] + A[i][k] * B[k][j]) % MOD
            return res
        
        def matrix_power(mat, power):
            result = [[1 if i == j else 0 for j in range(26)] for i in range(26)]
            while power > 0:
                if power % 2 == 1:
                    result = multiply(result, mat)
                mat = multiply(mat, mat)
                power //= 2
            return result
        
        T_power = matrix_power(T, t)
        
        # Multiply T_power with counts vector
        final = [0] * 26
        for i in range(26):
            for j in range(26):
                final[i] = (final[i] + T_power[i][j] * counts[j]) % MOD
        
        return sum(final) % MOD
    
assert Solution().lengthAfterTransformations("abcyy",2,[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2]) == 7
assert Solution().lengthAfterTransformations("azbk",1,[2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]) == 8

91272833